In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.decomposition import LatentDirichletAllocation
from sentence_transformers import SentenceTransformer
from numpy.linalg import pinv
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self-signed certificate in certificate chain
[nltk_data]     (_ssl.c:1000)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self-signed certificate in certificate chain
[nltk_data]     (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self-signed certificate in certificate chain
[nltk_data]     (_ssl.c:1000)>


False

In [4]:
file_path = '/home/student/vishaka/collection_data.csv'
chunk_size = 10000
processed_chunks = []

In [5]:
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunk = chunk[['document_id', 'document_text']]
    processed_chunks.append(chunk)
collection_df = pd.concat(processed_chunks, ignore_index=True)

In [6]:
collection_df

,document_id,document_text
0,1,The Manhattan Project and its atomic bomb help...
1,2,Essay on The Manhattan Project - The Manhattan...
2,3,The Manhattan Project was the name for a proje...
3,4,versions of each volume as well as complementa...
4,5,The Manhattan Project. This once classified ph...
...,...,...
8841817,8841818,When metal salts emit short wavelengths of vis...
8841818,8841819,Thousands of people across the United States w...
8841819,8841820,"The recipe that creates blue, for example, inc..."
8841820,8841821,"On Independence Days of yore, old-timey crowds..."


In [ ]:
collection_df.shape

(8841822, 2)

In [ ]:
collection_df['document_text'][8]

"One of the main reasons Hanford was selected as a site for the Manhattan Project's B Reactor was its proximity to the Columbia River, the largest river flowing into the Pacific Ocean from the North American coast."

Preprocessing

In [7]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [36]:
# Initialize preprocessing tools
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [37]:
import re
from nltk.corpus import stopwords
import spacy

# Load SpaCy's English model
nlp = spacy.load('en_core_web_sm')

# Define preprocessing functions
def clean_text(text):
    """Clean text by removing special characters, numbers, and extra spaces."""
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = text.lower()  # Convert to lowercase
    return text

def remove_stopwords(text):
    """Remove stopwords from the text."""
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text_spacy(text):
    """Lemmatize text using SpaCy."""
    doc = nlp(text)  # Process text with SpaCy
    lemmatized_words = [token.lemma_ for token in doc if not token.is_punct]
    return ' '.join(lemmatized_words)

def preprocess_text_spacy(text):
    """Apply all preprocessing steps to the text using SpaCy for lemmatization."""
    text = clean_text(text)
    text = remove_stopwords(text)
    text = lemmatize_text_spacy(text)
    return text




In [38]:
collection_data_df=collection_df[:20000]

In [39]:
# Apply preprocessing to the 'document' column of the DataFrame
collection_data_df['processed_document'] = collection_data_df['document_text'].apply(preprocess_text_spacy)

/tmp/ipykernel_2464977/3035804518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collection_data_df['processed_document'] = collection_data_df['document_text'].apply(preprocess_text_spacy)


In [40]:

# Display the first few rows after preprocessing
print(collection_data_df[['document_text', 'processed_document']].head())

                                       document_text  \
0  The Manhattan Project and its atomic bomb help...   
1  Essay on The Manhattan Project - The Manhattan...   
2  The Manhattan Project was the name for a proje...   
3  versions of each volume as well as complementa...   
4  The Manhattan Project. This once classified ph...   

                                  processed_document  
0  manhattan project atomic bomb helped bring end...  
1  essay manhattan project manhattan project manh...  
2  manhattan project name project conduct world w...  
3  version volume well complementary website firs...  
4  manhattan project classify photograph feature ...  


In [10]:
import pandas as pd

# Read the TSV file
qrels_train_df = pd.read_csv('/home/student/vishaka/data/qrels.train.tsv', sep='\t')



In [ ]:
qrels_train_df

        1185869  0      0.1  1
0       1185868  0       16  1
1        597651  0       49  1
2        403613  0       60  1
3       1183785  0      389  1
4        312651  0      616  1
...         ... ..      ... ..
532755    19285  0  8841362  1
532756   558837  0  4989159  1
532757   559149  0  8841547  1
532758   706678  0  8841643  1
532759   405466  0  8841735  1

[532760 rows x 4 columns]


In [12]:
qrels_train_df.rename(columns={
    qrels_train_df.columns[0]: 'query_id',
    qrels_train_df.columns[1]: 'iteration',
    qrels_train_df.columns[2]: 'document_id',
    qrels_train_df.columns[3]: 'relevance_label'
}, inplace=True)


In [41]:
print(collection_data_df.head(20))

    document_id                                      document_text  \
0             1  The Manhattan Project and its atomic bomb help...   
1             2  Essay on The Manhattan Project - The Manhattan...   
2             3  The Manhattan Project was the name for a proje...   
3             4  versions of each volume as well as complementa...   
4             5  The Manhattan Project. This once classified ph...   
5             6  Nor will it attempt to substitute for the extr...   
6             7  Manhattan Project. The Manhattan Project was a...   
7             8  In June 1942, the United States Army Corps of ...   
8             9  One of the main reasons Hanford was selected a...   
9            10  group discussions, community boards or panels ...   
10           11  punishment designed to repair the damage done ...   
11           12  Tutorial: Introduction to Restorative Justice....   
12           13  Organize volunteer community panels, boards, o...   
13           14  The

In [44]:
# Step 1: Extract the document IDs from the first 20,000 rows of collection_data
doc_ids_within_20000 = collection_data_df[:20000]['document_id'].tolist()

# Step 2: Filter queries_train_df based on document IDs
filtered_queries = qrels_train_df[qrels_train_df['document_id'].isin(doc_ids_within_20000)]

# Display the result
print(filtered_queries.head())

# If you want to fetch only the query_id column, you can do this:
query_ids_within_20000 = filtered_queries['query_id'].unique()
print("Unique Query IDs:", query_ids_within_20000)


   query_id  iteration  document_id  relevance_label
0   1185868          0           16                1
1    597651          0           49                1
2    403613          0           60                1
3   1183785          0          389                1
4    312651          0          616                1
Unique Query IDs: [1185868  597651  403613 1183785  312651   80385  645590  645337  186154
  457407  441383  683408 1164799  484187  460668  666321  182487  564233
  455279  208108  733739 1164798  402608  443797  662502 1184679   14562
  602162  545059  708236  310130  693161  186617  573027 1173772  541973
  273090  441269  642237  503515  637443 1164796  749988  135841  295446
  653051  691147  410621 1164795  598443  596451  651441  452286  308543
  202126  114820  501778  531029  651110  594127 1164794  396032  705580
  658203  387734  655102  224712  411732 1164793  605902  581014  559240
  608711  535936  130335  147535 1164792  595576  569308  753706  627871
  67360

In [45]:
queries_train_df = pd.read_csv('/home/student/vishaka/data/queries.train.tsv', sep='\t')


In [46]:
print(queries_train_df.head(20))

     121352                                     define extreme
0    634306           what does chattel mean on credit history
1    920825            what was the great leap forward brainly
2    510633                tattoo fixers how much does it cost
3    737889                  what is decentralization process.
4    278900  how many cars enter the la jolla concours d' e...
5    674172                      what is a bank transit number
6    303205     how much can i contribute to nondeductible ira
7    570009         what are the four major groups of elements
8    492875                              sanitizer temperature
9     54528               blood clots in urine after menopause
10   203218                                   highmark address
11   473204     per sf cost in california for tenant build out
12   738368                             what is delta in2ition
13   507001              symptoms of an enlarged heart in dogs
14   466926       number of times congress voted to rep

In [47]:
queries_train_df.rename(columns={
    queries_train_df.columns[0]: 'query_id',
    queries_train_df.columns[1]: 'query'
}, inplace=True)


In [48]:
print(queries_train_df)

        query_id                                              query
0         634306           what does chattel mean on credit history
1         920825            what was the great leap forward brainly
2         510633                tattoo fixers how much does it cost
3         737889                  what is decentralization process.
4         278900  how many cars enter the la jolla concours d' e...
...          ...                                                ...
808725    633855             what does canada post regulations mean
808726   1059728                            wholesale lularoe price
808727    210839                      how can i watch the day after
808728    908165              what to use instead of pgp in windows
808729     50393     benefits of boiling lemons and drinking juice.

[808730 rows x 2 columns]


In [49]:
filtered_queries_df = queries_train_df[queries_train_df['query_id'].isin(query_ids_within_20000)]

# Display the result
print(filtered_queries_df)

        query_id                                              query
1513      582297                              what can protein make
3497      898717                  what std does metronidazole treat
4872      312651  how much does an average person make for tutoring
6564        7273                     What is a normal average pulse
6690     1003774  where was the flush toilet invented during the...
...          ...                                                ...
798094     17472            amsterdam average temperatures by month
798117    749988                            what is garlic used for
802702    685235         what is a good home remedy for hemorrhoids
805146    640439  what does it mean if you start bleeding during...
807232    837903                 what is the origin of father's day

[580 rows x 2 columns]


In [54]:
collection_data_df.to_csv('collection_data_df.csv')

In [55]:
qrels_train_df.to_csv('qrels_train_df.csv')

In [56]:
filtered_queries_df .to_csv('filtered_queries_df .csv')

In [57]:
num_extra_rows = 120
extra_noise_df = filtered_queries_df.sample(n=num_extra_rows, replace=True, random_state=42)
noisy_queries_df = pd.concat([filtered_queries_df, extra_noise_df], ignore_index=True)

In [58]:
noisy_queries_df

,query_id,query
0,582297,what can protein make
1,898717,what std does metronidazole treat
2,312651,how much does an average person make for tutoring
3,7273,What is a normal average pulse
4,1003774,where was the flush toilet invented during the...
...,...,...
695,924834,what windows version is this pc using
696,940456,when did qvc open
697,748182,what is fms medical term
698,1174916,is it george harrison's birthday


In [62]:
def cur_decomposition(X, rank):
    col_norms = np.sum(X**2, axis=0)
    row_norms = np.sum(X**2, axis=1)
    prob_cols = col_norms / np.sum(col_norms)
    prob_rows = row_norms / np.sum(row_norms)
    
    selected_cols = np.random.choice(X.shape[1], rank, replace=False, p=prob_cols)
    selected_rows = np.random.choice(X.shape[0], rank, replace=False, p=prob_rows)
    
    C = X[:, selected_cols]
    R = X[selected_rows, :]
    W = X[np.ix_(selected_rows, selected_cols)]
    U = np.linalg.pinv(W)
    return C, U, R

In [63]:
def lda_gibbs_sampling(docs, num_topics, num_iter=1000, alpha=0.1, beta=0.1):
    vocab = list(set(word for doc in docs for word in doc.split()))
    term_doc_matrix = np.zeros((len(docs), len(vocab)))
    
    for i, doc in enumerate(docs):
        for word in doc.split():
            term_doc_matrix[i, vocab.index(word)] += 1
    
    num_docs, vocab_size = term_doc_matrix.shape
    topic_assignments = np.random.randint(0, num_topics, size=(num_docs, vocab_size))
    
    doc_topic_counts = np.zeros((num_docs, num_topics))
    topic_word_counts = np.zeros((num_topics, vocab_size))
    topic_counts = np.zeros(num_topics)
    
    for d in range(num_docs):
        for w in range(vocab_size):
            topic = topic_assignments[d, w]
            doc_topic_counts[d, topic] += term_doc_matrix[d, w]
            topic_word_counts[topic, w] += term_doc_matrix[d, w]
            topic_counts[topic] += term_doc_matrix[d, w]
    
    for _ in range(num_iter):
        for d in range(num_docs):
            for w in range(vocab_size):
                word_count = term_doc_matrix[d, w]
                if word_count == 0:
                    continue
                
                topic = topic_assignments[d, w]
                doc_topic_counts[d, topic] -= word_count
                topic_word_counts[topic, w] -= word_count
                topic_counts[topic] -= word_count
                
                topic_probs = ((doc_topic_counts[d, :] + alpha) * (topic_word_counts[:, w] + beta) / (topic_counts + beta * vocab_size))
                topic_probs /= topic_probs.sum()
                
                new_topic = np.random.choice(num_topics, p=topic_probs)
                topic_assignments[d, w] = new_topic
                doc_topic_counts[d, new_topic] += word_count
                topic_word_counts[new_topic, w] += word_count
                topic_counts[new_topic] += word_count
    
    doc_topic_dist = (doc_topic_counts + alpha) / (doc_topic_counts.sum(axis=1, keepdims=True) + num_topics * alpha)
    topic_word_dist = (topic_word_counts + beta) / (topic_word_counts.sum(axis=1, keepdims=True) + vocab_size * beta)
    return doc_topic_dist, topic_word_dist, vocab

In [64]:
def dynamic_query_expansion(query, documents, model, top_n=3, original_weight=0.7, expanded_weight=0.3):
    unique_terms = set(' '.join(documents).split())
    term_embeddings = {term: model.encode(term) for term in unique_terms}
    
    query_terms = query.split()
    query_embeddings = [model.encode(term) for term in query_terms]
    query_embedding = np.mean(query_embeddings, axis=0)
    
    similarities = {term: cosine_similarity([query_embedding], [embedding])[0][0] for term, embedding in term_embeddings.items()}
    expanded_terms = sorted(similarities, key=similarities.get, reverse=True)[:top_n]
    
    expanded_embeddings = [term_embeddings[term] for term in expanded_terms]
    combined_embedding = original_weight * np.mean(query_embeddings, axis=0) + expanded_weight * np.mean(expanded_embeddings, axis=0)
    
    expanded_query_terms = set(query_terms).union(expanded_terms)
    return combined_embedding, ' '.join(expanded_query_terms)

In [65]:
def retrieve_documents_with_cur(query, documents, model, lda_topics, lda_vocab, rank=10, top_n=5):
    query_embedding, expanded_query = dynamic_query_expansion(query, documents, model)
    print(f"Expanded Query: '{expanded_query}'\n")
    
    document_embeddings = model.encode(documents)
    C, U, R = cur_decomposition(document_embeddings, rank=rank)
    reduced_document_embeddings = C @ U
    reduced_query_embedding = query_embedding @ R.T
    
    similarities = cosine_similarity([reduced_query_embedding], reduced_document_embeddings).flatten()
    top_indices = np.argsort(-similarities)[:top_n]
    
    print("LDA Topics Distribution for Top Documents:")
    for idx in top_indices:
        doc_topics = lda_topics[idx]
        print(f"Document: {documents[idx]}\nTopic Distribution: {doc_topics}\n")
    
    return top_indices